# Working with shapefiles and dataframe in Python Notebooks (based on Jupyter Notebooks) in ArcGIS

This code downloads an ArcGIS shapefile from an URL, unzips, reads, and converts to a pandas dataframe so that the data can be processed and visualized. 

In this exercise, we will work iwth the Drought Vulnerability Score data from Department of Water Resources, California. The data download link is:

https://gis.data.cnra.ca.gov/datasets/d539f06a037f4365b1535fbef6634f27_0.zip?outSR=%7B%22latestWkid%22%3A3857%2C%22wkid%22%3A102100%7D


In [ ]:
# Step 1. Download the data from a given URL and save it in local computer. 

import os
import requests

url = "https://gis.data.cnra.ca.gov/datasets/d539f06a037f4365b1535fbef6634f27_0.zip?outSR=%7B%22latestWkid%22%3A3857%2C%22wkid%22%3A102100%7D"
response = requests.get(url)
file_path = os.path.expanduser("G:\My Drive\Google_Drive_UTmail\Data Science\ArcGIS\DWR\DWR_project\Data\Water_Shortage_Score.zip")
open(file_path, "wb").write(response.content)

In [5]:
# Step 2. Unzip the downloaded folder. 

# Create a directory where the zipped file will be imported. 

import os
os.mkdir("G:\My Drive\Google_Drive_UTmail\Data Science\ArcGIS\DWR\DWR_project\Data\Water_Shortage_Score")

# Unzip and extract the foder content to the created directory

import zipfile
with zipfile.ZipFile("G:\My Drive\Google_Drive_UTmail\Data Science\ArcGIS\DWR\DWR_project\Data\Water_Shortage_Score.zip", 'r') as zip_ref:
    zip_ref.extractall("G:\My Drive\Google_Drive_UTmail\Data Science\ArcGIS\DWR\DWR_project\Data\Water_Shortage_Score")



In [7]:
# Step 3. Convert the feature class to raster. This step may not be necessary. 

import arcpy

# Specify your working geodatabase
workspace = r"G:\My Drive\Google_Drive_UTmail\Data Science\ArcGIS\DWR\DWR_project\DWR_Project.gdb"
arcpy.env.workspace = workspace

# Input feature class to convert
input_feature_class = "i07_Water_Shortage_Vulnerability_Sections"

# Output raster dataset
output_raster = "F2R_Total_Score"

# Field to use for rasterization (e.g., population, elevation, etc.)
field_to_rasterize = "Total_Scor"

# Perform the feature to raster conversion
arcpy.FeatureToRaster_conversion(input_feature_class, field_to_rasterize, output_raster)

print("Feature class converted to raster successfully.")


Feature class converted to raster successfully.


In [8]:
# Some useful tips, display all raster data in the existing geodatabase
import arcpy

# Set the workspace environment
arcpy.env.workspace = r"G:\My Drive\Google_Drive_UTmail\Data Science\ArcGIS\DWR\DWR_project\DWR_Project.gdb"

# Get a list of all rasters in the input folder
raster_classes = arcpy.ListRasters()
raster_classes


['F2R_Total_Score', 'Feature_i07_1', 'raster', 'raster_new', 'raster_new_1']

In [15]:
# Some more tips. Get a list of all feature in the input folder, currently there is none
feature_classes = arcpy.ListFeatureClasses()
feature_classes

[]

In [8]:
# More tips. Check the unit of the raster being used.

# Use arcpy.Describe() to get information about the raster
desc = arcpy.Describe(r"G:\My Drive\Google_Drive_UTmail\Data Science\ArcGIS\DWR\DWR_project\DWR_Project.gdb\F2R_Total_Score")
# desc will have all the information about the raster that you need
desc

catalogPath,G:\My Drive\Google_Drive_UTmail\Data Science\ArcGIS\DWR\DWR_project\DWR_Project.gdb\F2R_Total_Score
dataType,RasterDataset
bandCount,1
format,FGDBR
name (Projected Coordinate System),WGS_1984_Web_Mercator_Auxiliary_Sphere
factoryCode (WKID),3857
linearUnitName (Linear Unit),Meter
name (Geographic Coordinate System),GCS_WGS_1984
factoryCode (WKID),4326
angularUnitName (Angular Unit),Degree
datumName (Datum),D_WGS_1984


In [15]:
# Step 4. Finally convert the featureclass to a pandas dataframe
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor
df = pd.DataFrame.spatial.from_featureclass("G:\My Drive\Google_Drive_UTmail\Data Science\ArcGIS\DWR\DWR_project\Data\Water_Shortage_Score\i07_Water_Shortage_Vulnerability_Sections.shp")
df.head()

,FID,OBJECTID,COUNTY_CD,BASE_LN_ME,TOWNSHIP,RANGE,SECTION,MTRS,DomWellCou,DomWellDep,DomWellD_1,DomWellD_2,PrdWellCou,PrdWellDep,PrdWellD_1,PrdWellD_2,PubWellCou,PubWellDep,PubWellD_1,PubWellD_2,WCRFolderL,GlobalID,StateSmall,RC2a_Curre,RC2aa_Mult,rRC2aa_Mul,RC2d_Subsi,rRC2d_Subs,RC1a_Tempe,RC1c_Wildf,RC2b_CalFi,RC2e_Saltw,rRC1a_Temp,rRC1c_Wild,rRC2b_calf,rRC2c_Frac,rRC2e_Salt,rRC2f_Crit,rRC2g_Grou,rRC2i_SWRC,rRC2j_Perc,rRC3a_Well,RC2g_Min_W,RC2j_Farme,rRC2j_Pe_1,RC2f_Criti,RC2c_Fract,RC3a_Well_,rRC3c_FRA_,RC3c_FRA_D,RC5a_House,rRC5a_Hous,rRC1b_Sea_,Total_Scor,County_Nam,Comments,Source,Date_Last_,Last_Modif,Date_Data_,Shape__Are,Shape__Len,SHAPE
0,0,1,28,B,AY/,DEL,TA,BAY/DELTA,0,0,0,0,0,0,0,0,0,0,0,0,,{4FC81D9E-A6B6-4022-8204-AF1EC4D13D0E},No,0,2,0.4,0.0,0.0,2.454163,0.24487,Low Hazard,Yes,0.420065,0.099087,0.0,0,1,0,0,0.0,0.0,0.0,0.0,None,0.0,No,No,0,0.0,0,No,0,1,21.799802,Napa,,Polygons exported from Well Completion Reports...,2022-09-29,"AU, GDSS",2022-01-01,7.859067e+06,17378.785527,"{""rings"": [[[-13616240.946, 4606287.102899998]..."
1,1,2,49,B,AY/,DEL,TA,BAY/DELTA,0,0,0,0,0,0,0,0,0,0,0,0,,{284EC860-B6ED-41F3-9A93-2561D3CD7736},No,0,2,0.4,0.0,0.0,2.198486,0.24487,Low Hazard,Yes,0.420065,0.099087,0.0,0,1,0,0,0.0,0.0,0.0,0.0,None,0.0,No,No,0,0.0,0,No,0,1,21.799802,Sonoma,,Polygons exported from Well Completion Reports...,2022-09-29,"AU, GDSS",2022-01-01,6.115084e+07,43867.240030,"{""rings"": [[[-13626069.328499999, 4600982.1555..."
2,2,3,21,B,AY/,DEL,TA,BAY/DELTA,0,0,0,0,0,0,0,0,0,0,0,0,,{202E7677-6B87-47E9-86ED-FE6B2DE86A4A},No,0,2,0.4,0.0,0.0,2.028290,0.24487,Low Hazard,Yes,0.420065,0.099087,0.0,0,1,0,0,0.0,0.0,0.0,0.0,None,0.0,No,No,0,0.0,0,No,0,1,21.799802,Marin,,Polygons exported from Well Completion Reports...,2022-09-29,"AU, GDSS",2022-01-01,7.826795e+04,2060.280836,"{""rings"": [[[-13627743.7771, 4559727.109099999..."
3,3,4,43,B,AY/,DEL,TA,BAY/DELTA,0,0,0,0,0,0,0,0,0,0,0,0,,{6741BAAF-E522-448E-A84C-34C90A9393C6},No,0,2,0.4,0.0,0.0,2.211700,0.24487,Low Hazard,Yes,0.420065,0.099087,0.0,0,1,0,0,0.0,0.0,0.0,0.0,None,0.0,No,No,0,0.0,0,No,0,1,21.799802,Santa Clara,,Polygons exported from Well Completion Reports...,2022-09-29,"AU, GDSS",2022-01-01,1.982955e+07,34383.739708,"{""rings"": [[[-13593772.648, 4504167.336400002]..."
4,4,5,34,B,AY/,DEL,TA,BAY/DELTA,0,0,0,0,0,0,0,0,0,0,0,0,,{F5A5FB24-06B6-434E-98CC-2E32CFABE867},No,0,2,0.4,0.0,0.0,2.635742,0.24487,Low Hazard,Yes,0.420065,0.099087,0.0,0,1,0,0,0.0,0.0,0.0,0.0,None,0.0,No,No,0,0.0,0,No,0,1,21.799802,Sacramento,,Polygons exported from Well Completion Reports...,2022-09-29,"AU, GDSS",2022-01-01,5.211623e+07,164911.173529,"{""rings"": [[[-13543057.3136, 4605213.704499997..."


In [16]:
# Step 5. Perform the required calculations. 

# Now lets calculate the maximum value of Total_Scor
max_score = df['Total_Scor'].max()
print(max_score)
county_max_score = df

100.055221


In [17]:
# Let's find out the County Name that has the maximum score
df[df.Total_Scor == max_score].County_Nam

83190    San Luis Obispo
Name: County_Nam, dtype: string

In [18]:
# Now lets find all the unique counties that have the Total Score greater than 95. 
# Note that the drop_duplicates displays unique counties only. 
df[df.Total_Scor > 95].County_Nam.drop_duplicates()

8101            Tuolumne
30900             Madera
37137             Fresno
41523               Lake
61038              Kings
67066             Tulare
83190    San Luis Obispo
Name: County_Nam, dtype: string